In [2]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.7.0


Hyperparameter
  - Batch Size
  - Define the class names

In [3]:
BATCH_SIZE = 32 
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [4]:
print(BATCH_SIZE)
print(classes)

32
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


## Loading and Preprocessing Data
[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset has 32 x 32 RGB images belonging to 10 classes. You will load the dataset from Keras.




In [5]:
(training_images, training_labels),(validation_images,validation_labels) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 3s 0us/step


In [6]:
print(training_images.shape,training_labels.shape)

(50000, 32, 32, 3) (50000, 1)


In [7]:
print(validation_images.shape,validation_labels.shape)

(10000, 32, 32, 3) (10000, 1)


## Format input to correspond to the pretrained model.

In [8]:
def preprocess_image_input(input_images):
  input_images = input_images.astype("float32")
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims

In [9]:
train_x = preprocess_image_input(training_images)
valid_x = preprocess_image_input(validation_images)

## Developing the Model

In [10]:
# Define Feature Extractor

In [11]:
def feature_extractor(inputs):
  feature_extractor_layer = tf.keras.applications.resnet.ResNet50(
      input_shape = (224,224,3),
      include_top = False, 
      weights = "imagenet")(inputs)

  return feature_extractor_layer

In [12]:
# define classifier
def classifier(inputs):
  x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(1024, activation = "relu")(x)
  x = tf.keras.layers.Dense(512, activation = "relu")(x)
  x = tf.keras.layers.Dense(10, activation = "softmax", name = "classification")(x)

  return x

In [13]:
# Finalize model
def final_model(inputs):
  resize = tf.keras.layers.UpSampling2D(size = (7,7))(inputs)
  resnet_feature_extractor = feature_extractor(resize)
  classification_output = classifier(resnet_feature_extractor)
  
  return classification_output


In [14]:
# define compile model
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  classification_output = final_model(inputs)
  model = tf.keras.Model(inputs = inputs, outputs = classification_output)
  model.compile(optimizer = "SGD",
                loss = "sparse_categorical_crossentropy",
                metrics = ["accuracy"])
  return model


In [15]:
model = define_compile_model()

94781440/94765736 [==============================] - 0s 0us/step


In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              209817

## Train the Model

In [17]:
EPOCHS = 4
history = model.fit(train_x,
                    training_labels,
                    epochs = EPOCHS,
                    validation_data = (valid_x, validation_labels),
                    batch_size = 64)

Epoch 1/4
782/782 [==============================] - 271s 331ms/step - loss: 0.3973 - accuracy: 0.8702 - val_loss: 0.1942 - val_accuracy: 0.9303
Epoch 2/4
782/782 [==============================] - 257s 328ms/step - loss: 0.0988 - accuracy: 0.9677 - val_loss: 0.2000 - val_accuracy: 0.9323
Epoch 3/4
782/782 [==============================] - 257s 329ms/step - loss: 0.0319 - accuracy: 0.9910 - val_loss: 0.1826 - val_accuracy: 0.9446
Epoch 4/4
782/782 [==============================] - 257s 328ms/step - loss: 0.0129 - accuracy: 0.9971 - val_loss: 0.1616 - val_accuracy: 0.9532
